In [ ]:
# import subprocess
# import os

# result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
# output = result.stdout
# for line in output.splitlines():
#     if '=' in line:
#         var, value = line.split('=', 1)
#         os.environ[var] = value

In [ ]:
# %mkdir -p modelsave checkpoint/
# %pip install --force-reinstall /home/z1933/workplace/machinelearning/PII-Data-Detection/unidecode/Unidecode-1.3.8-py3-none-any.whl

In [ ]:
from cfg import *
from util import *
Config.modelsavepath

In [ ]:
##### main
data = json.load(open(Config.data_path+"/train.json"))

# downsampling of negative examples
# p=[] # positive samples (contain relevant labels)
# n=[] # negative samples (presumably contain entities that are possibly wrongly classified as entity)
# for d in data:
#     if any(np.array(d["labels"]) != "O"): p.append(d)
#     else: n.append(d)
# print("original datapoints: ", len(data))

external = json.load(open(Config.data_path+"/mixtral-8x7b-v1.json"))
print("external datapoints: ", len(external))

# moredata = json.load(open(Config.data_path+"/moredata_dataset_fixed.json"))
# print("moredata datapoints: ", len(moredata))

data = data+external
# print("combined: ", len(data))


In [ ]:
data_pd=pd.DataFrame(data)
len(data_pd[data_pd['tokens'].apply(len)>2024])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(Config.model_name)
tokenized_input =tokenizer(data_pd.iloc[data_pd.full_text.str.len().idxmax()]['full_text'],return_tensors='pt')

In [ ]:
# tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokenized_input["input_ids"].size()

In [ ]:
from itertools import chain
#将所有数据的labels连接在一起,然后查重,转成list的格式,然后从小到大排序
all_labels = sorted(list(set(chain(*[x["labels"] for x in data]))))
print(f"all_labels:{all_labels}")
label2id = {l: i for i,l in enumerate(all_labels)}
#这个是{id:label},上面是{label:id}
id2label = {v:k for k,v in label2id.items()}

# Open a file for writing
with open(Config.modelsavepath+"//idlabel.json", "w") as f:
    # Write the map to the file in JSON format
    json.dump({'label2id':label2id,'id2label':id2label}, f)
id2label

In [ ]:
# EDA
train_df = data
print(f"len(train_df):{len(train_df)}, train_df[0].keys(): {list(train_df[0].keys())}")
print("-"*50)


print(f"train_df length:", len(train_df))

labels = set()
label_counts = {}
for i in range(len(train_df)):
    labels.update(train_df[i]['labels'])
    for label in train_df[i]['labels']:
        if label in label_counts:
            label_counts[label] += 1
        else:
            label_counts[label] = 1
            
print(f"labels: {labels}")
print('-'*25)
print(f"label_counts: {label_counts}")

In [ ]:

# # EDA
# display bar
# import seaborn as sns
# import copy
# display_label=copy.copy(label_counts)
# del  display_label['O']
# y =  [i for i in display_label.values()]

# g=sns.barplot(x=list(display_label.keys()),y=y)
# g.set_xticklabels(g.get_xticklabels(), rotation=90)

In [ ]:
tmp=dict()
for i in train_df:
    for j in range(0,len(i['tokens'])):
        if not i['tokens'][j].isascii():
            if i['tokens'][j] in tmp:
                tmp[i['tokens'][j]]+=1
            else:
                tmp[i['tokens'][j]]=0
dict(sorted(tmp.items(), key=lambda x: x[1],reverse=True))

In [ ]:
df_train=pd.DataFrame(train_df)
df_train_tokens=[]
for row in df_train.itertuples(index=False):
    for i in range(len(row.tokens)):
        df_train_tokens.append((row.document,row.tokens[i],row.labels[i],row.trailing_whitespace[i]))
df_train_tokens=pd.DataFrame(df_train_tokens,columns=['document','tokens','labels','trailing_whitespace'])


In [ ]:
df_train_tokens.loc[(df_train_tokens['tokens']=='Geoff')]

In [ ]:
input_data=pd.DataFrame(data)

In [ ]:
# DEA
# print all unicode character
# def has_unicode_codepoints(text):
#     return any(ord(char) > 127 for char in text)
# tmp_set=set()
# pattern =re.compile(r"[^\u0000-\u00ff]")
# for i in data:
#     for j in i['tokens']:
#         if has_unicode_codepoints(j):
#             tmp_set.add(j)
# for i in tmp_set:
#     print(i)

In [ ]:
# !cp ./nltk_data/* ~/nltk_data/ -r

In [ ]:
ds = Dataset.from_dict({
    "full_text": [x["full_text"] for x in data],
    "document": [str(x["document"]) for x in data],
    "tokens": [x["tokens"] for x in data],
    "trailing_whitespace": [x["trailing_whitespace"] for x in data],
    "labels": [x["labels"] for x in data]
})


预测时，空缺值填'O'

In [ ]:
# EDA
# import matplotlib.pyplot as plt
# import seaborn as sns
# tmp_list=[]
# for doc in data:
#     full_text=doc['full_text']
#     split_lines=full_text.split('\n\n')
#     for i in split_lines:
#         tmp_list.append(len(tokenizer(i)['input_ids']))
# print(len(tmp_list))
# print(max(tmp_list))
# sns.displot(tmp_list)

In [ ]:
# 根据'\n\n'分token
preprocesssed_ds=ds.map(train_preprocesss, fn_kwargs={'tokenizer':tokenizer,'label2id':label2id},num_proc=Config.num_proc,desc="prepocessing data")
preprocesssed_ds

In [ ]:
tmp_str="."
tmp_res=tokenizer(tmp_str)
tmp_res

In [ ]:
tokenizer.convert_ids_to_tokens(tmp_res['input_ids'])

In [ ]:
# rebuid dataset 
tmp_pd=expanddataset(preprocesssed_ds)
print(tmp_pd['berttokenids'].str.len().agg(['mean','max','std','min']))
print("len after expand",len(tmp_pd))

In [ ]:
tmp_pd.loc[tmp_pd['berttokenids'].str.len()==20]

In [ ]:
full_ds=Dataset.from_pandas(tmp_pd)
print(full_ds)

In [ ]:
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader,random_split
from transformers import DataCollatorForTokenClassification

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_len=int(0.75*len(full_ds)) # 长度分割，兼容老版本torch
train_dataset, val_dataset = random_split(full_ds, [train_len,len(full_ds)-train_len],generator=torch.Generator().manual_seed(Config.seed)) #TODO:改变比例


In [ ]:
print(f"train dataset len:{len(train_dataset)}")
print(f"vail dataset len:{len(val_dataset)}")

In [ ]:
data_collator = Collate(tokenizer=tokenizer)
train_dataloader=DataLoader(train_dataset,batch_size=Config.batch_size,pin_memory=True,collate_fn=data_collator,shuffle=True)
val_dataloader=DataLoader(val_dataset,batch_size=Config.batch_size,pin_memory=True,collate_fn=data_collator)

In [ ]:
from model1 import NLPModel
model=NLPModel(id2label,label2id,Config.model_name).to(device)
from torch.optim import AdamW
from transformers import get_cosine_schedule_with_warmup
from seqeval.metrics import recall_score, precision_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score
import gc

optimizer= AdamW(model.parameters(),lr=Config.lr,weight_decay=Config.weight_decay)

scaler = torch.cuda.amp.GradScaler()
valstep = len(train_dataloader) //Config.evaltimes

num_train_steps = int(len(train_dataloader) * Config.epochs)
scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=Config.num_warmup_steps, num_training_steps=num_train_steps, 
            )

In [ ]:
def compute_metrics(predictions,traget, all_labels):
    
    # Remove ignored index (special tokens)
    true_predictions = [
        [all_labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, traget)
    ]
    true_labels = [
        [all_labels[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, traget)
    ]
    
    recall = recall_score(true_labels, true_predictions)
    precision = precision_score(true_labels, true_predictions)
    f1_score = (1 + 5*5) * recall * precision / (5*5*precision + recall)
    
    results = {
        'recall': recall,
        'precision': precision,
        'f1': f1_score
    }
    return results

def validation(val_dataloader,model):
    model.eval()

    allprecitions=[]
    alltargets=[]
    for step,dataset in enumerate(tqdm(val_dataloader)):

        with torch.no_grad():
            ids = dataset["ids"].to(device,non_blocking=True)
            mask = dataset["mask"].to(device,non_blocking=True)
            targets = dataset["targets"].to(device,non_blocking=True)
            tokentype = dataset["type_ids"].to(device,non_blocking=True)
            logit,loss = model(ids,mask, tokentype,targets)

            targets=targets.view(-1)
            targets = targets.detach().cpu().numpy()

            allprecitions.append(np.argmax(logit, axis=1))
            alltargets.append(targets)
        del ids,mask,targets,tokentype,loss,logit

    score=compute_metrics(allprecitions,alltargets,all_labels)
    
    gc.collect()
    torch.cuda.empty_cache()

    model.train()

    return score

In [ ]:
tokenizer.save_pretrained(Config.modelsavepath)

In [ ]:
gc.collect()
torch.cuda.empty_cache()
start_time=time.time()
bestscore=0
epoch=0

if Config.resume_train_epoch:
    checkpoint = torch.load(Config.workdir+f"//checkpoint//checkpoint{Config.resume_train_epoch}.pth")
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    scaler.load_state_dict(checkpoint['scaler_state_dict'])
    epoch = checkpoint['epoch']
    bestscore=checkpoint['bestscore']
    model.train()
    print(f"start epoch:{epoch}")

while epoch <Config.epochs:
    losses=[] 
    for step,dataset in enumerate(tqdm(train_dataloader)):
        with torch.cuda.amp.autocast():
            ids = dataset["ids"].to(device,non_blocking=True)
            mask = dataset["mask"].to(device,non_blocking=True)
            tokentype = dataset["type_ids"].to(device,non_blocking=True)
            targets = dataset["targets"].to(device,non_blocking=True)

            logit,loss = model(ids,mask, tokentype,targets)
            loss/= Config.accumulation_steps

        # Accumulates scaled gradients.
        scaler.scale(loss).backward()
        # losses.append(loss.item())
        
        if (step+1) % Config.accumulation_steps ==0:
            scaler.step(optimizer)
            scale = scaler.get_scale()
            # Updates the scale for next iteration.
            scaler.update()
            skip_lr_sched = (scale != scaler.get_scale())
            if not skip_lr_sched:
                scheduler.step()
            optimizer.zero_grad()

        # validation
        if epoch>0 and (step+1) % valstep == 0:
            scores=validation(val_dataloader,model)
            print("Validation:Epoch{} Step [{}/{}] Loss: {} Time: {:.1f} Metric: {}".format(epoch,step, len(train_dataloader)-1, loss.item(), time.time()-start_time,scores))
            if scores['f1']>bestscore:
                print(f"Best score is {bestscore} → {scores['f1']}. Saving model")
                bestscore=scores['f1']
                torch.save(model.state_dict(), os.path.join(Config.modelsavepath,"model_seed{}_score{:.3f}.pth".format(Config.seed,bestscore)))
            else:
                print("no improvement, bestf1 is {:.3f} score is {}".format(bestscore,scores))
                # report loss
        if (step+1) % Config.logging_steps ==0:
            # print("Logging: Step [{}/{}] Loss: {:.3f} Time: {:.1f}".format(step, len(train_dataloader)-1, loss.item(), time.time()-start_time))
            gc.collect()
            torch.cuda.empty_cache()
        del ids,mask,targets,tokentype,loss,logit

    epoch+=1
    # checkpoint
    torch.save({
            'epoch': epoch,
            'bestscore':bestscore,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict':scheduler.state_dict(),
            'scaler_state_dict':scaler.state_dict()
            }, Config.modelsavepath+"/checkpoint{}.pth".format(epoch))

tokenizer.save_pretrained(Config.modelsavepath)
os.system("/usr/bin/shutdown")